In [15]:
%pip install torch
%pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [1]:
DATA_TRAIN_PROCESSED = '../data/prepocessed/train.csv'

# Import

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
from statsmodels.tsa.forecasting.theta import ThetaModel
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

from sklearn.linear_model import LinearRegression
import xgboost as xgb


from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
import torch.nn as nn

# Define

## Preprocessing

In [17]:
def normalize_data(data, method="minmax"):
    """Chuẩn hóa dữ liệu theo phương pháp MinMax hoặc StandardScaler."""
    if method == "minmax":
        scaler = MinMaxScaler(feature_range=(0, 1))
    elif method == "standard":
        scaler = StandardScaler()
    else:
        return data, None

    data_scaled = scaler.fit_transform(np.array(data).reshape(-1, 1))
    return data_scaled, scaler


def create_sequences(data, lookback):
    """Tạo dữ liệu theo dạng (X, y) cho các model dùng sliding window."""
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

## Model

In [18]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

In [19]:
def train_lstm(X_train, y_train, epochs=50):
    """Huấn luyện mô hình LSTM."""
    model = LSTMModel(input_dim=1, hidden_dim=50, num_layers=2, output_dim=1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    return model

In [25]:
def train_arima(train_data):
    """Huấn luyện mô hình ARIMA."""
    model = auto_arima(train_data, seasonal=False, stepwise=True, suppress_warnings=True, trace=True)
    return model

def train_sarima(train_data, seasonality=12):
    """Huấn luyện mô hình SARIMA (có yếu tố mùa vụ)."""
    model = auto_arima(train_data, seasonal=True, m=seasonality, stepwise=True, suppress_warnings=True, trace=True)
    return model

In [21]:
def train_theta(train_data, seasonality=12):
    """Huấn luyện mô hình Theta."""
    return ThetaModel(train_data, period=seasonality).fit()

In [22]:
def train_xgboost(X_train, y_train):
    """Huấn luyện mô hình XGBoost."""
    model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
    model.fit(X_train.reshape(X_train.shape[0], -1), y_train)
    return model

In [23]:
def train_linear(X_train, y_train):
    """Huấn luyện mô hình Linear Regression."""
    model = LinearRegression()
    model.fit(X_train.reshape(X_train.shape[0], -1), y_train)
    return model

## Pipeline

In [41]:
def forecast_pipeline(data, model_type="LSTM", normalize="minmax", lookback=10, **kwargs):
    """
    Pipeline huấn luyện và dự báo chuỗi thời gian với nhiều model.
    
    Parameters:
        - data: pd.Series (chuỗi thời gian)
        - model_type: str (["LSTM", "ARIMA", "SARIMA", "Theta", "XGBoost", "Linear"])
        - normalize: str (["minmax", "standard", "none"])
        - lookback: int (số bước nhìn lại cho mô hình LSTM, XGBoost, Linear Regression)
        - kwargs: Các tham số bổ sung cho từng loại model
            + seasonality: chỉ yêu cầu cho SARIMA và Theta
    """
    # ________________________________________ CHECK INPUT _______________________________________ #
    if not isinstance(data, pd.DataFrame) or "Date" not in data.columns or "temperatures" not in data.columns:
        raise ValueError("Input data must be a DataFrame with columns ['Date', 'temperatures']")

    # Chuyển cột Date thành datetime nếu chưa đúng định dạng
    data["Date"] = pd.to_datetime(data["Date"])
    data.set_index("Date", inplace=True)
    #_________________________________________PREPROCESS_______________________________________#
    
    data_scaled, scaler = normalize_data(data, method=normalize)
    split_idx = int(len(data) * 0.8)
    train_data, test_data = data_scaled[:split_idx], data_scaled[split_idx:]

    if model_type in ["LSTM", "XGBoost", "Linear"]:
        X_train, y_train = create_sequences(train_data, lookback)
        X_test, y_test = create_sequences(test_data, lookback)

    #_________________________________________TRAIN MODEL______________________________________#
    conf_int = None  
    if model_type == "LSTM":
        model = train_lstm(X_train, y_train)
        X_test = torch.tensor(X_test, dtype=torch.float32)
        with torch.no_grad():
            y_pred = model(X_test).numpy()

    elif model_type == "ARIMA":
        model = train_arima(train_data)
        forecast_result = model.predict(n_periods=len(test_data), return_conf_int=True)
        y_pred, conf_int = forecast_result[0], forecast_result[1]

    elif model_type == "SARIMA":
        if "seasonality" not in kwargs:
            raise ValueError("SARIMA requires 'seasonality' parameter.")
        model = train_sarima(train_data, seasonality=kwargs["seasonality"])
        forecast_result = model.predict(n_periods=len(test_data), return_conf_int=True)
        y_pred, conf_int = forecast_result[0], forecast_result[1]

    elif model_type == "Theta":
        if "seasonality" not in kwargs:
            raise ValueError("Theta model requires 'seasonality' parameter.")
        model = train_theta(train_data, seasonality=kwargs["seasonality"])
        y_pred = model.forecast(steps=len(test_data), return_conf_int=True)
        conf_int = y_pred.conf_int()

    elif model_type == "XGBoost":
        model = train_xgboost(X_train, y_train)
        y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))

    elif model_type == "Linear":
        model = train_linear(X_train, y_train)
        y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))

    else:
        raise ValueError("Unsupported model type. Choose from ['LSTM', 'ARIMA', 'SARIMA', 'Theta', 'XGBoost', 'Linear'].")

    #_____________________________________EVALUATE___________________________________________#
    if scaler:
        y_pred_actual = scaler.inverse_transform(y_pred.reshape(-1, 1))
        y_test_actual = scaler.inverse_transform(test_data[:len(y_pred)].reshape(-1, 1))
        if conf_int is not None:
            lower_bound = scaler.inverse_transform(conf_int[:, 0].reshape(-1, 1))
            upper_bound = scaler.inverse_transform(conf_int[:, 1].reshape(-1, 1))
            conf_int = (lower_bound, upper_bound)
    else:
        y_pred_actual, y_test_actual = y_pred, test_data[:len(y_pred)]
    
    plot_forecast(y_test_actual, y_pred_actual, model_type, normalize, conf_int)

In [35]:
def plot_forecast(y_test_actual, y_pred_actual, model_type, normalize, conf_int=None):
    """
    Vẽ biểu đồ dự báo chuỗi thời gian kèm vùng tin cậy nếu có.
    
    Parameters:
        - y_test_actual: Giá trị thực tế
        - y_pred_actual: Giá trị dự báo
        - model_type: Tên mô hình
        - normalize: Loại chuẩn hóa
        - conf_int: Tuple (lower_bound, upper_bound) cho vùng tin cậy
    """
    plt.figure(figsize=(12, 6))
    
    # Vẽ giá trị thực tế
    plt.plot(y_test_actual, label="Actual", color='blue')
    
    # Vẽ giá trị dự báo
    plt.plot(y_pred_actual, label="Predicted", color='red', linestyle='dashed')
    
    # Vẽ vùng tin cậy (nếu có)
    if conf_int is not None:
        lower_bound, upper_bound = conf_int
        plt.fill_between(range(len(y_pred_actual)), lower_bound, upper_bound, color='pink', alpha=0.3, label="Confidence Interval")
    
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.title(f"Forecasting with {model_type} (Normalization: {normalize})")
    plt.show()

# Run

In [36]:
data = pd.read_csv(DATA_TRAIN_PROCESSED)

In [39]:
data

,Date,temperatures
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8
...,...,...
2915,1988-12-26,9.5
2916,1988-12-27,12.9
2917,1988-12-28,12.9
2918,1988-12-29,14.8


In [42]:
forecast_pipeline(data=data, model_type="SARIMA", normalize="none", seasonality=12)
forecast_pipeline(data=data, model_type="Theta", normalize="standard", seasonality=12)
forecast_pipeline(data=data, model_type="Linear", normalize="minmax", lookback=24)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=3.84 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=13230.398, Time=0.03 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=11151.379, Time=0.92 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=11726.709, Time=0.53 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=18175.193, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=11162.695, Time=0.14 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=11146.205, Time=2.32 sec
 ARIMA(1,0,0)(2,0,1)[12] intercept   : AIC=inf, Time=5.32 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.31 sec
 ARIMA(0,0,0)(2,0,0)[12] intercept   : AIC=12261.742, Time=1.98 sec
 ARIMA(2,0,0)(2,0,0)[12] intercept   : AIC=11141.972, Time=3.11 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=11144.136, Time=1.13 sec
 ARIMA(2,0,0)(2,0,1)[12] intercept   : AIC=inf, Time=6.76 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.85 sec
 ARIMA(3,0,0)(2,0,0)[12] intercept 

C:\Users\Admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


AttributeError: 'NoneType' object has no attribute 'inverse_transform'